# チャットモデルのファインチューニング方法

ファインチューニングは、プロンプトに収まる数よりもはるかに多くの例でトレーニングすることでモデルを改善し、幅広いタスクでより良い結果を得ることができます。このノートブックでは、新しいGPT-4o miniファインチューニングのステップバイステップガイドを提供します。[RecipeNLGデータセット](https://github.com/Glorf/recipenlg)を使用してエンティティ抽出を実行します。このデータセットは様々なレシピと、それぞれから抽出された汎用的な材料のリストを提供します。これは固有表現認識（NER）タスクでよく使用されるデータセットです。

注意：**GPT-4o miniファインチューニングは、[Tier 4および5の使用階層](https://platform.openai.com/docs/guides/rate-limits/usage-tiers)の開発者が利用できます。** ファインチューニングダッシュボードにアクセスし、「create」をクリックして、ベースモデルのドロップダウンから「gpt-4o-mini-2024-07-18」を選択することで、GPT-4o miniのファインチューニングを開始できます。

以下のステップを実行します：

1. **セットアップ：** データセットを読み込み、ファインチューニングする1つのドメインに絞り込みます。
2. **データ準備：** トレーニングと検証の例を作成し、`Files`エンドポイントにアップロードして、ファインチューニング用のデータを準備します。
3. **ファインチューニング：** ファインチューニングされたモデルを作成します。
4. **推論：** 新しい入力に対してファインチューニングされたモデルを使用して推論を行います。

これらを完了すると、`gpt-4o-mini-2024-07-18`モデルのトレーニング、評価、デプロイができるようになります。

ファインチューニングの詳細については、[ドキュメントガイド](https://platform.openai.com/docs/guides/fine-tuning)または[APIリファレンス](https://platform.openai.com/docs/api-reference/fine-tuning)を参照してください。

## セットアップ

In [6]:
# make sure to use the latest version of the openai python package
!pip install --upgrade --quiet openai

In [69]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    organization="<org id>",
    project="<project id>",
)

ファインチューニングは特定のドメインに焦点を当てた場合に最も効果的に機能します。データセットがモデルが学習するのに十分集中していながら、未知の例を見逃さないよう十分に一般的であることを確認することが重要です。これを念頭に置いて、RecipesNLGデータセットから[cookbooks.com](https://cookbooks.com/)のドキュメントのみを含むサブセットを抽出しました。

In [70]:
# Read in the dataset we'll use for this task.
# This will be the RecipesNLG dataset, which we've cleaned to only contain documents from www.cookbooks.com
recipe_df = pd.read_csv("data/cookbook_recipes_nlg_10k.csv")

recipe_df.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


## データ準備

まずはデータの準備から始めます。`ChatCompletion`形式でファインチューニングを行う場合、各トレーニング例は単純な`messages`のリストになります。例えば、エントリは以下のようになります：

```
[{'role': 'system',
  'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},

 {'role': 'user',
  'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},

 {'role': 'assistant',
  'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]
```

トレーニングプロセス中、この会話は分割され、最後のエントリがモデルが生成する`completion`となり、残りの`messages`がプロンプトとして機能します。トレーニング例を構築する際はこの点を考慮してください - モデルがマルチターンの会話で動作する場合は、会話が拡張し始めた際にパフォーマンスが低下しないよう、代表的な例を提供してください。

現在、各トレーニング例には4096トークンの制限があることにご注意ください。これより長いものは4096トークンで切り捨てられます。

In [71]:
system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."


def create_user_message(row):
    return f"Title: {row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients: "


def prepare_example_conversation(row):
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": create_user_message(row)},
            {"role": "assistant", "content": row["NER"]},
        ]
    }


pprint(prepare_example_conversation(recipe_df.iloc[0]))

{'messages': [{'content': 'You are a helpful recipe assistant. You are to '
                          'extract the generic ingredients from each of the '
                          'recipes provided.',
               'role': 'system'},
              {'content': 'Title: No-Bake Nut Cookies\n'
                          '\n'
                          'Ingredients: ["1 c. firmly packed brown sugar", '
                          '"1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 '
                          'c. broken nuts (pecans)", "2 Tbsp. butter or '
                          'margarine", "3 1/2 c. bite size shredded rice '
                          'biscuits"]\n'
                          '\n'
                          'Generic ingredients: ',
               'role': 'user'},
              {'content': '["brown sugar", "milk", "vanilla", "nuts", '
                          '"butter", "bite size shredded rice biscuits"]',
               'role': 'assistant'}]}


データセットのサブセットを使用してトレーニングデータとして活用してみましょう。30-50個の適切に精選された例から始めることができます。トレーニングセットのサイズを増やすにつれてパフォーマンスが線形にスケールし続けることが確認できるはずですが、ジョブの実行時間も長くなります。

In [72]:
# use the first 100 rows of the dataset for training
training_df = recipe_df.loc[0:100]

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: Jewell Ball\'S Chicken\n\nIngredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["bee

訓練データに加えて、**オプションで**検証データを提供することもできます。これは、モデルが訓練セットに対してオーバーフィットしないことを確認するために使用されます。

In [73]:
validation_df = recipe_df.loc[101:200]
validation_data = validation_df.apply(
    prepare_example_conversation, axis=1).tolist()

次に、データを`.jsonl`ファイルとして保存する必要があります。各行が1つの訓練例の会話となります。

In [74]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [75]:
training_file_name = "tmp_recipe_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

これが私たちの訓練用`.jsonl`ファイルの最初の5行の内容です：

In [76]:
# print the first 5 lines of the training file
!head -n 5 tmp_recipe_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: No-Bake Nut Cookies\n\nIngredients: [\"1 c. firmly packed brown sugar\", \"1/2 c. evaporated milk\", \"1/2 tsp. vanilla\", \"1/2 c. broken nuts (pecans)\", \"2 Tbsp. butter or margarine\", \"3 1/2 c. bite size shredded rice biscuits\"]\n\nGeneric ingredients: "}, {"role": "assistant", "content": "[\"brown sugar\", \"milk\", \"vanilla\", \"nuts\", \"butter\", \"bite size shredded rice biscuits\"]"}]}
{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: Jewell Ball'S Chicken\n\nIngredients: [\"1 small jar chipped beef, cut up\", \"4 boned chicken breasts\", \"1 can cream of mushroom soup\", \"1 carton sour cream\"]\n\nGeneric ingredients: "}, {"role":

### ファイルのアップロード

ファインチューニングされたモデルで使用するために、`Files`エンドポイントにファイルをアップロードできるようになりました。

In [77]:
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


training_file_id = upload_file(training_file_name, "fine-tune")
validation_file_id = upload_file(validation_file_name, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-3wfAfDoYcGrSpaE17qK0vXT0
Validation file ID: file-HhFhnyGJhazYdPcd3wrtvIoX


## ファインチューニング

生成されたファイルとモデルを識別するためのオプションのサフィックスを使用して、ファインチューニングジョブを作成できます。レスポンスには`id`が含まれており、これを使用してジョブの更新情報を取得できます。

注意：ファイルは最初にシステムで処理される必要があるため、`File not ready`エラーが発生する場合があります。その場合は、数分後に再試行してください。

In [81]:
MODEL = "gpt-4o-mini-2024-07-18"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    suffix="recipe-ner",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-UiaiLwGdGBfdLQDBAoQheufN
Status: validating_files


#### ジョブステータスの確認

`https://api.openai.com/v1/alpha/fine-tunes`エンドポイントに`GET`リクエストを送信して、アルファファインチューニングジョブの一覧を取得できます。この場合、前のステップで取得したIDが最終的に`status: succeeded`になっていることを確認する必要があります。

完了したら、`result_files`を使用して検証セット（アップロードした場合）から結果をサンプリングし、`fine_tuned_model`パラメータのIDを使用してトレーニング済みモデルを呼び出すことができます。

In [91]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-UiaiLwGdGBfdLQDBAoQheufN
Status: running
Trained Tokens: None


fine-tuneの進行状況は、eventsエンドポイントで追跡できます。fine-tuneの準備が完了するまで、以下のセルを数回再実行できます。

In [94]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 288/303: training loss=0.00
Step 289/303: training loss=0.01
Step 290/303: training loss=0.00, validation loss=0.31
Step 291/303: training loss=0.00
Step 292/303: training loss=0.00
Step 293/303: training loss=0.00
Step 294/303: training loss=0.00
Step 295/303: training loss=0.00
Step 296/303: training loss=0.00
Step 297/303: training loss=0.00
Step 298/303: training loss=0.01
Step 299/303: training loss=0.00
Step 300/303: training loss=0.00, validation loss=0.04
Step 301/303: training loss=0.16
Step 302/303: training loss=0.00
Step 303/303: training loss=0.00, full validation loss=0.33
Checkpoint created at step 101 with Snapshot ID: ft:gpt-4o-mini-2024-07-18:openai-gtm:recipe-ner:9o1eNlSa:ckpt-step-101
Checkpoint created at step 202 with Snapshot ID: ft:gpt-4o-mini-2024-07-18:openai-gtm:recipe-ner:9o1eNFnj:ckpt-step-202
New fine-tuned model created: ft:gpt-4o-mini-2024-07-18:openai-gtm:recipe-ner:9o1eNNKO
The job has successfully completed


これで完了したので、ジョブからファインチューニングされたモデルIDを取得できます：

In [95]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError(
        "Fine-tuned model ID not found. Your job has likely not been completed yet."
    )

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-4o-mini-2024-07-18:openai-gtm:recipe-ner:9o1eNNKO


## 推論

最後のステップは、ファインチューニングしたモデルを推論に使用することです。従来の`FineTuning`と同様に、`model`パラメータに新しいファインチューニングしたモデル名を指定して`ChatCompletions`を呼び出すだけです。

In [96]:
test_df = recipe_df.loc[201:300]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'generic ingredients from each of the recipes provided.',
  'role': 'system'},
 {'content': 'Title: Beef Brisket\n'
             '\n'
             'Ingredients: ["4 lb. beef brisket", "1 c. catsup", "1 c. water", '
             '"1/2 onion, minced", "2 Tbsp. cider vinegar", "1 Tbsp. prepared '
             'horseradish", "1 Tbsp. prepared mustard", "1 tsp. salt", "1/2 '
             'tsp. pepper"]\n'
             '\n'
             'Generic ingredients: ',
  'role': 'user'}]


In [97]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

["beef brisket", "catsup", "water", "onion", "cider vinegar", "horseradish", "mustard", "salt", "pepper"]


## まとめ

おめでとうございます！これで`ChatCompletion`形式を使用して独自のモデルをファインチューニングする準備が整いました！あなたが構築するものを楽しみにしています。